In [1]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.weatheri.co.kr/forecast/forecast08.php'
header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
res = requests.get(url, headers=header, verify=False)
res.encoding = 'utf-8'
res.text[:500]

c:\Users\young\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.weatheri.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


'\n\n\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml">\n<html lang="ko">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<title>생활지수 : 웨더아이 - 날씨로부터의 자유, 날씨와 함께하는 행복</title>\n<LINK REL="stylesheet" TYPE="text/css" HREF="/images/style.css">\n<script src="/images/AC_RunActiveContent.js" language="javascript"></script>\n<style type="text/css">\n<!--\nbody {\n\tmargin'

In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()     # PATH 설정 할 것 (chromedriver.exe가 위치해있는곳을 path 설정해야함)
driver.get(url)

In [42]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('tbody > tr > td > font > strong')
len(trs)

12

In [43]:
trs.encoding = 'utf-8'
trs

[<strong>서울/경기지역 </strong>,
 <strong>빨래지수 : 60</strong>,
 <strong>외출지수 : 70</strong>,
 <strong>운동지수 : 60</strong>,
 <strong>세차지수 : 30</strong>,
 <strong>우산지수 : 20</strong>,
 <strong>수면지수 : 80</strong>,
 <strong>계절별 생활지수 </strong>,
 <strong>피부지수 
 																		: 80</strong>,
 <strong>감기지수 : 70</strong>,
 <strong>불조심지수 :80</strong>,
 <strong>난방지수  
 																		:10</strong>]

In [45]:
ment =driver.find_element(By.XPATH,'/html/body/table[2]/tbody/tr[3]/td[2]/table/tbody/tr[3]/td/table/tbody/tr/td/table/tbody/tr[1]/td[2]').text
ment
# 다른 내용이 출력.....와이~~~~~~~~~~~

'빨래지수 : 60\n바깥 날씨 주의하세요'

In [48]:
ment =driver.find_element(By.XPATH,'/html/body/table[2]/tbody/tr[3]/td[2]/table/tbody/tr[3]/td/table/tbody/tr/td/table/tbody/tr[1]/td[4]').text
ment

'외출지수 : 70\n외출 약속도 좋아요'

In [16]:
# 생활지수
tr = trs[0]
#img = tr.select_one('tbody > tr > td > img')['src']
name = tr.select_one('tbody > tr > td > font > strong')
index = tr.select_one('tbody > tr > td > font > strong')
ment = tr.select_one('tbody > tr > td > font')
name, index, ment

AttributeError: 'NoneType' object has no attribute 'get_text'

In [14]:
data = []
from datetime import datetime, timedelta

for tr in trs:
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    img = tr.select_one('img')['src']
    item = tr.select_one('h1').get_text().strip().replace('.', '')
    index = tr.select_one('h2').get_text().strip()
    ment = tr.select_one('p').get_text().strip()


    url = 'https://www.kr-weathernews.com/mv3/html/lifeindex.html?region='
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    data.append({ '이미지': img, '생활지수': item, '지수': index, '안내멘트': ment})

In [15]:
df = pd.DataFrame(data)
df

,이미지,생활지수,지수,안내멘트
0,../img/icon_idx_carwash.png?ver=20231116,세차지수,30,오늘 세차는 다시 한번 생각해 보세요
1,../img/icon_idx_running.png?ver=20231116,달리기지수,70,달리기에 나쁘지 않은 날씨예요
2,../img/icon_idx_washing.png?ver=20231116,빨래지수,26,야외에서 빨래 말리기는 절대 NO
3,../img/icon_idx_sleep.png?ver=20231116,수면지수,30,잠자기에 조금 힘든 날이에요
4,../img/icon_idx_umb.png?ver=20231116,우산지수,30,혹시 모르니 가벼운 우산을 챙겨보세요


In [18]:
# 
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs2 = soup.select('#slidePage2 > #pointList > li')
len(trs2)

2

In [20]:
trs2.encoding = 'utf-8'
trs2

[<li><img alt="자외선지수 사진" src="../img/icon_life_uv.png?ver=20231116"/><h1>자외선지수</h1><h2> 3</h2><p>야외활동 안심하고 마음껏 하세요</p><progress max="100" value="25"></progress></li>,
 <li><img alt="감기지수 사진" src="../img/icon_life_cold.png?ver=20231116"/><h1>감기지수</h1><h2> 3</h2><p>노약자는 외부 활동을 되도록 삼가요</p><progress max="100" value="75"></progress></li>]

In [21]:
# 생활지수
tr2 = trs2[0]
img = tr2.select_one('img')['src']
item = tr2.select_one('h1').get_text().strip().replace('.', '')
index = tr2.select_one('h2').get_text().strip()
ment = tr2.select_one('p').get_text().strip()
item, index, ment

('자외선지수', '3', '야외활동 안심하고 마음껏 하세요')

In [22]:
for tr2 in trs2:
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    img = tr2.select_one('img')['src']
    item = tr2.select_one('h1').get_text().strip().replace('.', '')
    index = tr2.select_one('h2').get_text().strip()
    ment = tr2.select_one('p').get_text().strip()


    url = 'https://www.kr-weathernews.com/mv3/html/lifeindex.html?region='
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    data.append({ '이미지': img, '생활지수': item, '지수': index, '안내멘트': ment})

In [24]:
df = pd.DataFrame(data)
df

,이미지,생활지수,지수,안내멘트
0,../img/icon_idx_carwash.png?ver=20231116,세차지수,30,오늘 세차는 다시 한번 생각해 보세요
1,../img/icon_idx_running.png?ver=20231116,달리기지수,70,달리기에 나쁘지 않은 날씨예요
2,../img/icon_idx_washing.png?ver=20231116,빨래지수,26,야외에서 빨래 말리기는 절대 NO
3,../img/icon_idx_sleep.png?ver=20231116,수면지수,30,잠자기에 조금 힘든 날이에요
4,../img/icon_idx_umb.png?ver=20231116,우산지수,30,혹시 모르니 가벼운 우산을 챙겨보세요
5,../img/icon_life_uv.png?ver=20231116,자외선지수,3,야외활동 안심하고 마음껏 하세요
6,../img/icon_life_cold.png?ver=20231116,감기지수,3,노약자는 외부 활동을 되도록 삼가요
